In [1]:
import tensornetwork as tn
import numpy as np

In [25]:
tn.Node(np.array([[[1.0 + 0.0j], [0.0 + 0.0j]], [[1.0 + 0.0j], [0.0 + 0.0j]]]))

Node
(
name : '__unnamed_node__',
tensor : 
array([[1.+0.j],
       [0.+0.j],
       [0.+0.j]]),
edges : 
[
Edge(Dangling Edge)[0] 
, 
Edge(Dangling Edge)[1] 
] 
)

In [19]:
state = tn.Node(np.array([1.0 + 0.0j, 0.0 + 0.0j]))
qubit_in = state[0]

# This node represents the Hadamard gate we wish to perform
# on this qubit.
hadamard = tn.Node(np.array([[1, 1], [1, -1]])) / np.sqrt(2)
tn.connect(qubit_in, hadamard[0]) # Equal to qubit ^ hadamard[0]
# The "output edge" of the operation represents the qubit after
#  applying the operation.
qubit_out = hadamard[1]
# Contraction is how you actually "apply" the gate.
state2 = state @ hadamard
print(state2.tensor) # array([0.707+0.j, 0.707+0.j])

[0.70710678+0.j 0.70710678+0.j]


In [20]:
state

Node
(
name : '__unnamed_node__',
tensor : 
array([1.+0.j, 0.+0.j]),
edges : 
[
Edge(Dangling Edge)[0] 
] 
)

In [21]:
state2

Node
(
name : '__unnamed_node__',
tensor : 
array([0.70710678+0.j, 0.70710678+0.j]),
edges : 
[
Edge(Dangling Edge)[0] 
] 
)

In [22]:
hadamard

Node
(
name : '__unnamed_node__',
tensor : 
array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]]),
edges : 
[
Edge(Dangling Edge)[0] 
, 
Edge(Dangling Edge)[1] 
] 
)